In [12]:
pip install requests PyMuPDF langchain faiss-cpu sentence-transformers numpy cohere langchain-community

Note: you may need to restart the kernel to use updated packages.


In [14]:
# Import libraries
import requests, fitz, os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import numpy as np, faiss, getpass
from langchain.llms import Cohere

# Download IPC PDF
ipc_url = "https://legislative.gov.in/sites/default/files/A1860-45.pdf"
pdf_path = "ipc.pdf"
if not os.path.exists(pdf_path):
    try:
        response = requests.get(ipc_url)
        with open(pdf_path, 'wb') as f:
            f.write(response.content)
        print("IPC PDF downloaded.")
    except Exception as e:
        print(f"Download error: {e}")
        exit()

# Extract text from PDF
try:
    pdf = fitz.open(pdf_path)
    ipc_text = "".join(page.get_text() for page in pdf)
    pdf.close()
    with open('IPC_text.txt', 'w', encoding="utf-8") as f:
        f.write(ipc_text)
    print("Text extracted.")
except Exception as e:
    print(f"PDF extraction error: {e}")
    exit()

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(ipc_text)
print(f"Text split into {len(texts)} chunks.")

# Generate embeddings and create FAISS index
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts, convert_to_tensor=True)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings.cpu().numpy())
print("FAISS index created.")

# Initialize Cohere model
COHERE_API_KEY = getpass.getpass("Enter Cohere API Key: ")
os.environ["COHERE_API_KEY"] = COHERE_API_KEY
llm = Cohere(model="command", temperature=0.7)

# Chatbot response function
def get_response(query):
    query_embedding = model.encode([query], convert_to_tensor=True)
    distances, indices = index.search(query_embedding.cpu().numpy(), k=1)
    context = texts[indices[0][0]]
    prompt = f"IPC section: {context}\nQuestion: {query}\nAnswer concisely based on the IPC section."
    return llm(prompt)

# Interactive loop
if __name__ == "__main__":
    while True:
        query = input("Ask about the Indian Penal Code (type 'exit' to quit): ")
        if query.lower() == 'exit':
            break
        print(get_response(query))

Text extracted.
Text split into 3 chunks.
FAISS index created.
 Sorry, the provided text does not discuss the specifics of the Indian Penal Code. Hence, I cannot provide an accurate calculation of how many pages the Indian Penal Code contains. 
If you have a copy of the code itself, perhaps you could scan or screenshot a few pages, and I would be delighted to estimate the total number of pages computationally for you. 
IPC 298 states that if any person voluntarily causes hurt by dangerous weapons or instigates others to do so, they shall be punished with imprisonment for either a term of three years, or with a fine, or with both. 
IPC section 297 discusses "injuring or defiling place of worship with intent to insult the religion of any class". This covers the crime of purposely damaging, defacing, or desecrating a place of worship, or sacred object related to religion, with the intent to insult or outrage the religious beliefs of a group of people. It's important to note that this clau